In [1]:
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin
import pandas as pd

/Users/mahmoud/anaconda3/envs/learn-env/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.0.3) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
def get_car_data(car_url):
    response = requests.get(car_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    review_section = soup.find('div', {'class': 'vehicle-reviews'})
    if review_section:
        reviews = review_section.find_all('div', {'class': 'review-section'})
        review_texts = [review.get_text().strip() for review in reviews]
    else:
        review_texts = []

    rating_element = soup.find('span', {'class': 'sds-rating__count'})
    car_rating = rating_element.get_text() if rating_element else 'No rating available'

    return review_texts, car_rating

car_reviews_data = []

In [3]:
for i in range(11, 19):
    url = f'https://www.cars.com/shopping/results/?dealer_id=&keyword=&list_price_max=15000&list_price_min=&makes[]=&maximum_distance=20&mileage_max=&monthly_payment=256&page='+str(i)+'&page_size=20&sort=best_match_desc&stock_type=all&year_max=&year_min=&zip='

    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    car_links_depth = soup.find_all("a", {"class": "vehicle-card-link js-gallery-click-link"})

    for car_link in car_links_depth:
        car_url = urljoin(url, car_link.get('href'))  # finds the full URL automatically
        car_name = car_link.find('h2', {'class': 'title'}).get_text().strip()
        car_reviews, car_rating = get_car_data(car_url)

        car_reviews_data.append({'Car Name': car_name, 'Rating': car_rating, 'Reviews': car_reviews})

In [5]:
ten_reviews_df = pd.DataFrame(car_reviews_data)
ten_reviews_df

,Car Name,Rating,Reviews
0,2012 Honda Accord EX-L,2.6,"[September 20, 2022\nBy Manny from Milford, CT..."
1,2014 Chevrolet Impala 1LS,4.6,"[May 10, 2023\nBy Glenda Jones from Warner Ro..."
2,2001 Mitsubishi Eclipse Spyder GT,3.6,"[March 11, 2023\nBy Anthony from Bakersfield.C..."
3,2002 Toyota Avalon XLS,3.6,"[December 21, 2022\nBy John A Smith from Milwa..."
4,1990 Volkswagen Corrado Base,3.3,"[May 24, 2018\nBy jeffsgolife from franklin\nO..."
...,...,...,...
155,2012 Chevrolet Cruze LS,5.0,"[February 12, 2023\nBy Discouraged Mammaw from..."
156,2010 Lexus RX 350 Base,4.7,"[September 21, 2022\nBy Jose Ferrer from Miam..."
157,2007 Lexus ES 350,5.0,"[November 10, 2022\nBy Ann from Mobile, Al\nOw..."
158,2007 Lexus RX 400h Base (CVT),4.0,"[March 19, 2021\nBy Carol K. from West Allis, ..."
